In [1]:
import time
import schedule
import pandas as pd
from decimal import Decimal, ROUND_DOWN
from binance.client import Client
from binance.enums import SIDE_BUY, SIDE_SELL, ORDER_TYPE_LIMIT, ORDER_TYPE_MARKET

# =========================================================
# BINANCE CLIENT (SPOT TESTNET)
# =========================================================
# ✅ DO NOT hardcode keys in code. Use env vars in real use.
API_KEY = "ENTER_YOUR_KEY_HERE"
API_SECRET = "ENTER_YOUR_SECRET_HERE"

client = Client(API_KEY, API_SECRET, testnet=True)
client.API_URL = "https://testnet.binance.vision/api"

# =========================================================
# CONFIG
# =========================================================
SYMBOL = "ETHUSDT"
BASE_ASSET = "ETH"
QUOTE_ASSET = "USDT"

TIMEFRAME = Client.KLINE_INTERVAL_1MINUTE
LIMIT = 55

# Strategy params
TP_PERCENT = 0.30
SL_PERCENT = 0.25
CONSOLIDATION_PERCENT = 0.7

ENTRY_TIMEOUT_SEC = 60
MONITOR_EVERY_SEC = 2
ENTRY_EVERY_SEC = 20

# ✅ Risk-based sizing + caps
RISK_PCT = 0.01          # 1% equity risk per trade (tweak: 0.25%–0.5% recommended)
CAPITAL_CAP_FRAC = 0.50   # cap: max 50% of equity used as notional

# ✅ Safety
MIN_QTY_OVERRIDE = None   # set to a float if you want a hard minimum qty (else uses exchange filters)
MAX_QTY_OVERRIDE = None   # set to a float if you want a hard maximum qty
COMMISSION_BUFFER = 1.002 # buffer for fees/slippage when checking quote balance

# =========================================================
# SYMBOL FILTERS (tick/step/minNotional/minQty/maxQty)
# =========================================================
def get_symbol_filters(symbol: str):
    info = client.get_symbol_info(symbol)
    if not info:
        raise ValueError(f"Symbol not found: {symbol}")

    filters = {f["filterType"]: f for f in info["filters"]}

    tick_size = Decimal(filters["PRICE_FILTER"]["tickSize"])
    step_size = Decimal(filters["LOT_SIZE"]["stepSize"])

    min_qty = Decimal(filters["LOT_SIZE"]["minQty"])
    max_qty = Decimal(filters["LOT_SIZE"]["maxQty"])

    min_notional = Decimal("0")
    if "MIN_NOTIONAL" in filters:
        min_notional = Decimal(filters["MIN_NOTIONAL"].get("minNotional", "0"))

    return tick_size, step_size, min_qty, max_qty, min_notional


TICK_SIZE, STEP_SIZE, EX_MIN_QTY, EX_MAX_QTY, EX_MIN_NOTIONAL = get_symbol_filters(SYMBOL)


def round_price(p: float) -> str:
    d = (Decimal(str(p)) / TICK_SIZE).to_integral_value(rounding=ROUND_DOWN) * TICK_SIZE
    return format(d, "f")


def round_qty(q: float) -> str:
    d = (Decimal(str(q)) / STEP_SIZE).to_integral_value(rounding=ROUND_DOWN) * STEP_SIZE
    return format(d, "f")


# =========================================================
# DATA FUNCTIONS
# =========================================================
def get_candles(symbol: str, interval: str, limit: int) -> pd.DataFrame:
    klines = client.get_klines(symbol=symbol, interval=interval, limit=limit)

    df = pd.DataFrame(klines, columns=[
        "open_time", "open", "high", "low", "close", "volume",
        "close_time", "qav", "num_trades", "taker_base", "taker_quote", "ignore"
    ])

    for col in ["open", "high", "low", "close", "volume"]:
        df[col] = df[col].astype(float)

    return df.reset_index(drop=True)


def get_bid_ask(symbol: str):
    ob = client.get_order_book(symbol=symbol, limit=5)
    bids = ob.get("bids", [])
    asks = ob.get("asks", [])

    if not bids or not asks:
        last_price = float(client.get_symbol_ticker(symbol=symbol)["price"])
        return last_price, last_price

    bid = float(bids[0][0])
    ask = float(asks[0][0])
    return bid, ask


def get_free_balance(asset: str) -> float:
    bal = client.get_asset_balance(asset=asset)
    return float(bal["free"]) if bal and bal.get("free") is not None else 0.0


# =========================================================
# INDICATORS / CONSOLIDATION WINDOW
# =========================================================
def calc_tr(df: pd.DataFrame) -> float:
    high = df["high"]
    low = df["low"]
    close = df["close"]
    prev_close = close.shift(1)

    tr_series = pd.concat([
        (high - low),
        (high - prev_close).abs(),
        (low - prev_close).abs()
    ], axis=1).max(axis=1)

    df["True_Range"] = tr_series
    return float(tr_series.iloc[-1])


def get_extreme_of_consolidation(df: pd.DataFrame, percent: float):
    if "True_Range" not in df.columns:
        _ = calc_tr(df)

    for i in range(len(df) - 1, -1, -1):
        tr = float(df["True_Range"].iloc[i])
        close = float(df["close"].iloc[i])
        tr_dev = (tr / close) * 100 if close else 0.0

        if tr_dev > percent:
            window = df.iloc[i + 1:]
            if len(window) == 0:
                break
            return float(window["low"].min()), float(window["high"].max())

    return float(df["low"].min()), float(df["high"].max())


# =========================================================
# ORDER / STATE HELPERS
# =========================================================
def has_open_orders(symbol: str) -> bool:
    return len(client.get_open_orders(symbol=symbol)) > 0


def get_order_status(symbol: str, order_id: int):
    return client.get_order(symbol=symbol, orderId=order_id)


def cancel_order(symbol: str, order_id: int):
    try:
        client.cancel_order(symbol=symbol, orderId=order_id)
    except Exception:
        pass


def _reset_state_after_exit():
    state["in_position"] = False
    state["tp"] = None
    state["sl"] = None
    state["entry_order_id"] = None
    state["entry_time"] = None
    state["entry_price"] = None
    state["position_qty"] = 0.0
    print("[STATE] Reset after exit.")


# =========================================================
# RISK-BASED SIZING
# =========================================================
def compute_qty_risk_based(entry_price: float, sl_price: float) -> float:
    """
    qty = (equity * risk_pct) / (entry - stop)
    then cap by max notional = equity * CAPITAL_CAP_FRAC
    then enforce min/max qty + minNotional + balance availability
    """
    quote_free = get_free_balance(QUOTE_ASSET)
    base_free = get_free_balance(BASE_ASSET)

    # Equity approximation for spot: quote + base*price
    equity_quote = quote_free + base_free * entry_price

    stop_dist = entry_price - sl_price
    if stop_dist <= 0:
        return 0.0

    risk_dollars = equity_quote * RISK_PCT
    raw_qty = risk_dollars / stop_dist

    # Capital cap (notional cap)
    max_notional = equity_quote * CAPITAL_CAP_FRAC
    cap_qty = max_notional / entry_price
    qty = min(raw_qty, cap_qty)

    # Exchange min/max qty
    min_qty = float(EX_MIN_QTY)
    max_qty = float(EX_MAX_QTY)

    if MIN_QTY_OVERRIDE is not None:
        min_qty = max(min_qty, float(MIN_QTY_OVERRIDE))
    if MAX_QTY_OVERRIDE is not None:
        max_qty = min(max_qty, float(MAX_QTY_OVERRIDE))

    # Clamp before rounding
    qty = max(0.0, min(qty, max_qty))
    if qty < min_qty:
        return 0.0

    # Round down to step size
    qty_rounded = float(round_qty(qty))

    # Re-check after rounding
    if qty_rounded < min_qty:
        return 0.0

    # Min notional check
    notional = qty_rounded * entry_price
    if float(EX_MIN_NOTIONAL) > 0 and notional < float(EX_MIN_NOTIONAL):
        return 0.0

    # Balance check (quote balance for buy)
    cost_est = notional * COMMISSION_BUFFER
    if cost_est > quote_free:
        # Try scale down to what we can afford
        affordable_qty = (quote_free / COMMISSION_BUFFER) / entry_price
        affordable_qty = float(round_qty(affordable_qty))
        if affordable_qty < min_qty:
            return 0.0
        # minNotional re-check
        if float(EX_MIN_NOTIONAL) > 0 and affordable_qty * entry_price < float(EX_MIN_NOTIONAL):
            return 0.0
        return affordable_qty

    return qty_rounded


# =========================================================
# STATE
# =========================================================
state = {
    "in_position": False,
    "tp": None,
    "sl": None,
    "entry_order_id": None,
    "entry_time": None,
    "entry_price": None,
    "position_qty": 0.0,
}


# =========================================================
# LOOP A: ENTRY LOOP (risk-based)
# =========================================================
def entry_loop():
    print(f"[HEARTBEAT] entry_loop scanning @ {time.strftime('%H:%M:%S')}")

    try:
        if state["in_position"]:
            print("[SKIP] already in position")
            return

        # If an entry order is pending, don't place another
        if state["entry_order_id"] is not None:
            print("[SKIP] entry pending")
            return

        # If exchange shows open orders, don't stack
        if has_open_orders(SYMBOL):
            print("[SKIP] open orders exist on exchange")
            return

        candles = get_candles(SYMBOL, TIMEFRAME, LIMIT)
        tr = calc_tr(candles)
        last_close = float(candles["close"].iloc[-1])
        tr_deviance = (tr / last_close) * 100 if last_close else 0.0

        bid, ask = get_bid_ask(SYMBOL)
        low, high = get_extreme_of_consolidation(candles, CONSOLIDATION_PERCENT)
        rng = high - low
        lower_third = low + (rng / 3)

        print(
            f"[SCAN] bid={bid:.2f} ask={ask:.2f} | "
            f"TR%={tr_deviance:.3f} (th={CONSOLIDATION_PERCENT}) | "
            f"range_low={low:.2f} range_high={high:.2f} lower_third={lower_third:.2f}"
        )

        # Only trade when "consolidating": TR% below threshold
        if tr_deviance >= CONSOLIDATION_PERCENT:
            print("[NO] Not consolidating -> skip")
            return

        # Entry condition: bid in lower third
        if bid > lower_third:
            print("[NO] Price not in lower third -> skip")
            return

        entry_ref = bid
        tp = entry_ref * (1 + TP_PERCENT / 100)
        sl = entry_ref * (1 - SL_PERCENT / 100)

        qty = compute_qty_risk_based(entry_ref, sl)
        if qty <= 0:
            print("[SKIP] risk sizing produced qty=0 (minQty/minNotional/balance/caps)")
            return

        print(
            f"[ENTRY] BUY signal | entry={entry_ref:.2f} | "
            f"TP={tp:.2f} SL={sl:.2f} | "
            f"qty={qty:.6f} | notional≈{qty*entry_ref:.2f} {QUOTE_ASSET}"
        )

        order = client.create_order(
            symbol=SYMBOL,
            side=SIDE_BUY,
            type=ORDER_TYPE_LIMIT,
            timeInForce="GTC",
            quantity=round_qty(qty),
            price=round_price(entry_ref)
        )

        state["entry_order_id"] = order.get("orderId")
        state["entry_time"] = time.time()
        state["entry_price"] = entry_ref
        state["tp"] = tp
        state["sl"] = sl
        state["position_qty"] = float(qty)

        print("[ENTRY] Limit buy placed. OrderId:", state["entry_order_id"])

    except Exception as e:
        print("+++++ ENTRY LOOP ERROR:", e)
        time.sleep(5)


# =========================================================
# LOOP B: MONITOR LOOP (fill + TP/SL)
# =========================================================
def monitor_loop():
    try:
        bid, ask = get_bid_ask(SYMBOL)

        # Case 1: waiting for entry to fill
        if (not state["in_position"]) and state["entry_order_id"] is not None:
            oid = state["entry_order_id"]
            info = get_order_status(SYMBOL, oid)
            status = info.get("status")

            if status == "FILLED":
                executed_qty = float(info.get("executedQty", 0))
                state["in_position"] = True
                if executed_qty > 0:
                    state["position_qty"] = executed_qty

                print(f"[FILL] Entry filled. Qty={state['position_qty']} @ ~{state['entry_price']:.2f}")
                return

            elapsed = time.time() - state["entry_time"]
            if elapsed >= ENTRY_TIMEOUT_SEC:
                print("[ENTRY] Timeout. Cancelling entry order.")
                cancel_order(SYMBOL, oid)
                state["entry_order_id"] = None
                state["entry_time"] = None
                state["entry_price"] = None
                state["tp"] = None
                state["sl"] = None
                state["position_qty"] = 0.0
                return

            return

        # Case 2: in position -> check TP/SL
        if state["in_position"]:
            tp = state["tp"]
            sl = state["sl"]

            if tp is not None and bid >= tp:
                print(f"[EXIT] TP hit. bid={bid:.2f} >= {tp:.2f} -> MARKET SELL")
                client.create_order(
                    symbol=SYMBOL,
                    side=SIDE_SELL,
                    type=ORDER_TYPE_MARKET,
                    quantity=round_qty(state["position_qty"])
                )
                _reset_state_after_exit()
                return

            if sl is not None and bid <= sl:
                print(f"[EXIT] SL hit. bid={bid:.2f} <= {sl:.2f} -> MARKET SELL")
                client.create_order(
                    symbol=SYMBOL,
                    side=SIDE_SELL,
                    type=ORDER_TYPE_MARKET,
                    quantity=round_qty(state["position_qty"])
                )
                _reset_state_after_exit()
                return

    except Exception as e:
        print("+++++ MONITOR LOOP ERROR:", e)
        time.sleep(5)


# =========================================================
# SCHEDULING
# =========================================================
schedule.every(ENTRY_EVERY_SEC).seconds.do(entry_loop)
schedule.every(MONITOR_EVERY_SEC).seconds.do(monitor_loop)

print("Consolidation Pop bot running (RISK-BASED sizing + 50% capital cap, Spot Testnet)...")
print(f"Symbol={SYMBOL} | Risk={RISK_PCT*100:.2f}% | Cap={CAPITAL_CAP_FRAC*100:.0f}% | TP={TP_PERCENT}% | SL={SL_PERCENT}%")
print(f"Exchange filters: step={STEP_SIZE} tick={TICK_SIZE} minQty={EX_MIN_QTY} maxQty={EX_MAX_QTY} minNotional={EX_MIN_NOTIONAL}")

while True:
    try:
        schedule.run_pending()
        time.sleep(0.2)
    except Exception as e:
        print("+++++ MAIN LOOP ERROR, sleeping 30s:", e)
        time.sleep(30)


Consolidation Pop bot running (RISK-BASED sizing + 50% capital cap, Spot Testnet)...
Symbol=ETHUSDT | Risk=1.00% | Cap=50% | TP=0.3% | SL=0.25%
Exchange filters: step=0.00010000 tick=0.01000000 minQty=0.00010000 maxQty=9000.00000000 minNotional=0
[HEARTBEAT] entry_loop scanning @ 22:38:23
[SCAN] bid=2704.25 ask=2704.26 | TR%=0.110 (th=0.7) | range_low=2694.24 range_high=2709.90 lower_third=2699.46
[NO] Price not in lower third -> skip
[HEARTBEAT] entry_loop scanning @ 22:38:44
[SCAN] bid=2704.96 ask=2704.97 | TR%=0.110 (th=0.7) | range_low=2694.24 range_high=2709.90 lower_third=2699.46
[NO] Price not in lower third -> skip


KeyboardInterrupt: 